In [30]:
from typing import Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import  PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import  FewShotChatMessagePromptTemplate, FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector


    

chat = ChatOpenAI(temperature=0.1,
                   streaming=True, 
                   callbacks=[
                      StreamingStdOutCallbackHandler(),
                    ],
                  )

a_template = PromptTemplate.from_template("What is capital of {country}")
b_template = PromptTemplate(
  template="What is capital of {country}",
  input_variables=["country"],
)
a_template.format(country="Austria")
b_template.format(country="Austrailia")
# result = t | chat
# result.invoke({"country":"Austria"})

'What is capital of Austrailia'

In [32]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

example_prompt = PromptTemplate.from_template(
  "Human:{question} \n AI:{answer}"
)

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
  input_variables=["country",],
  suffix="Human:What do you know about {country}?"
)

# prompt.format(country="Turkey")
chain = prompt |chat
chain.invoke({
  "country":"Turkey"
})

AI:
        I know this:
        Capital: Ankara
        Language: Turkish
        Food: Kebabs and Baklava
        Currency: Turkish Lira

AIMessageChunk(content='AI:\n        I know this:\n        Capital: Ankara\n        Language: Turkish\n        Food: Kebabs and Baklava\n        Currency: Turkish Lira')

In [ ]:


example_prompt_chat = ChatPromptTemplate.from_messages([
  ("human", "What do you know about {question}?"),
  ("ai", "{answer}")
])

prompt_chat = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt_chat,
  examples=examples, 
)

final_prompt = ChatPromptTemplate.from_messages(
  [
  ("system","You are a geography expert"),
  prompt_chat,
  ("human", "What do you know about {country}?"),
])

chain = final_prompt |chat
chain.invoke({"country":"Israel"})


In [34]:

class RandomExampleSelector(BaseExampleSelector):
  def __init__(self, examples):
    self.examples = examples

  def add_example(self, example):
    self.examples.append(example)

  def select_examples(self, input_variables) :
    from random import choice
    return [choice(self.examples)]
  
  
example_prompt = PromptTemplate.from_template(
  "Human:{question} \n AI:{answer}"
)

example_selector = LengthBasedExampleSelector(
  examples=examples,
  example_prompt=example_prompt,
  max_length=380,
)

# 내가 만든거
example_selector_my = RandomExampleSelector(  
  examples = examples,
)

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  example_selector=example_selector_my,

  input_variables=["country",],
  suffix="Human:What do you know about {country}?"
)

prompt.format(country="Brazil")

TypeError: RandomExampleSelector.__init__() got an unexpected keyword argument 'examples'